In [2]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections, copy, pickle
from termcolor import colored
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [3]:
import scipy.stats
from sklearn import preprocessing
import keras
import keras.utils
from keras import optimizers, backend as K
from keras.layers import *
from keras.models import Model

Using TensorFlow backend.


In [4]:
import util.plot
import util.data

In [5]:
reload(util.plot)
reload(util.data);

In [6]:
# with open('data/encoder.pkl', 'rb') as f:
#     E = pickle.load(f)

In [7]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=1*1000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=1000)
data.shape, data.srch_id.unique().size

((1000, 79), 40)

In [8]:
for k in data.columns:
    if data[k].isna().sum() > 0:
        print('rm ', k)
        data.drop(columns=[k], inplace=True)

varsUsed = list(data.columns)
# varsUsed.remove('Unnamed')
# varsUsed.remove('position')
# varsUsed.remove('random_bool')
len(varsUsed)

rm  orig_destination_distance
rm  comp1_rate
rm  comp1_inv
rm  comp1_rate_percent_diff
rm  comp2_rate
rm  comp2_inv
rm  comp2_rate_percent_diff
rm  comp3_rate
rm  comp3_inv
rm  comp3_rate_percent_diff
rm  comp4_rate
rm  comp4_inv
rm  comp4_rate_percent_diff
rm  comp5_rate
rm  comp5_inv
rm  comp5_rate_percent_diff
rm  comp6_rate
rm  comp6_inv
rm  comp6_rate_percent_diff
rm  comp7_rate
rm  comp7_inv
rm  comp7_rate_percent_diff
rm  comp8_rate
rm  comp8_inv
rm  comp8_rate_percent_diff
rm  travel_distance


53

In [9]:
data.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,hour,minute,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,score
0,1,0.0,8.0,0.098734,0.006029,0.0,893,-0.190084,-0.300346,1,...,8,32,0,0,0,0,1,0,0,0
1,1,0.0,8.0,0.098734,0.006029,0.0,10404,0.763816,0.191146,1,...,8,32,0,0,0,0,1,0,0,0
2,1,0.0,8.0,0.098734,0.006029,0.0,21315,-0.190084,0.682638,1,...,8,32,0,0,0,0,1,0,0,0
3,1,0.0,8.0,0.098734,0.006029,0.0,27348,-1.143983,0.191146,1,...,8,32,0,0,0,0,1,0,0,0
4,1,0.0,8.0,0.098734,0.006029,0.0,29604,0.763816,-0.300346,1,...,8,32,0,0,0,0,1,0,0,0


# AE 1) Search id

See also https://blog.keras.io/building-autoencoders-in-keras.html and https://github.com/voschezang/drum-style-transfer

In [10]:
# [k for k in data.columns if 'comp' not in k and 'prop' not in k]

In [11]:
# rm irrelevant info
keys_property = [k for k in data.columns if 'comp' in k or 'prop' in k]
keys_property += ['click_bool', 'booking_bool', 'gross_bookings_usd', 'random_bool', 'score', 'price_usd', 'position', 'travel_distance', 'travel_distances']
keys_search = [k for k in data.columns if k not in keys_property]
len(keys_search), keys_search

(35,
 ['srch_id',
  'site_id',
  'visitor_location_country_id',
  'visitor_hist_starrating',
  'visitor_hist_adr_usd',
  'promotion_flag',
  'srch_destination_id',
  'srch_length_of_stay',
  'srch_booking_window',
  'srch_adults_count',
  'srch_children_count',
  'srch_room_count',
  'srch_saturday_night_bool',
  'srch_query_affinity_score',
  'visitor_hist_starrating_is_null',
  'srch_person_per_room_score',
  'srch_adults_per_room_score',
  'srch_adults_count_float',
  'srch_children_count_float',
  'srch_room_count_float',
  'position_float',
  'srch_length_of_stay_float',
  'srch_booking_window_float',
  'year',
  'month',
  'day',
  'hour',
  'minute',
  'Friday',
  'Monday',
  'Saturday',
  'Sunday',
  'Thursday',
  'Tuesday',
  'Wednesday'])

In [12]:
data = data[keys_search]
data.srch_id.unique()
data_unique_srch_id = data.drop_duplicates(subset='srch_id')
assert data_unique_srch_id.shape[0] == data.srch_id.unique().size, \
    'if srch_id is equal, all non-property attributes should be equal as well'

In [13]:
assert data_unique_srch_id.srch_id.min() > 0, 'search id must be positive and nonzero'
assert ~data_unique_srch_id.srch_id.isna().any()
assert ~data_unique_srch_id.isna().any().any()
assert ~data_unique_srch_id.isin([np.nan, np.inf, -np.inf]).any().any()

In [18]:
# convert to np array & reshape
sample = data_unique_srch_id.sample(frac=1, random_state=seed)
x_train = sample.values[:,:,np.newaxis]
# x_train = data_unique_srch_id.values
# x_train = x_train.reshape(x_train.shape + (1,))
y_labels = x_train[:,0,:]
discretizer = preprocessing.KBinsDiscretizer(n_bins=y_labels.shape[0],
         encode='onehot', strategy='uniform')
y_train = discretizer.fit_transform(y_labels)
y_labels.shape, y_train.shape, x_train.shape

((40, 1), (40, 40), (40, 35, 1))

In [19]:
assert ~np.any(np.isinf(x_train))
assert ~np.any(np.isinf(-x_train))
assert ~np.any(np.isnan(x_train))

In [20]:
y_train = y_train.toarray()
y_train = y_train[:,:,np.newaxis]
y_train.shape

(40, 40, 1)

In [21]:
x_train = np.random.random(x_train.shape)
y_train = np.random.random(y_train.shape)

In [22]:
# n_dims = data.shape[1]
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]
print(input_shape, output_shape)
encoder_input = Input(shape=input_shape)
h = encoder_input
h = Flatten()(h)
h = Dense(100, activation='relu')(h)
# decoder
h = Dense(np.prod(output_shape), activation='relu')(h)
h = Reshape(output_shape)(h)
encoder_output = Activation('sigmoid')(h)
model = Model(encoder_input, encoder_output, name='ae-')
model.compile(optimizer='adam', loss='binary_crossentropy') # RMSprop Adadelta SGD

(35, 1) (40, 1)


In [23]:
# validation_split does not shuffle the data
result = model.fit(x_train, y_train, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
# history = result.history

Train on 32 samples, validate on 8 samples
Epoch 1/2
32/32 [==============================] - 1s 23ms/step - loss: 0.6963 - val_loss: 0.7009
Epoch 2/2
32/32 [==============================] - 0s 232us/step - loss: 0.6938 - val_loss: 0.6991


In [24]:
# result.params
result.history

{'val_loss': [0.7009005546569824, 0.6990842819213867],
 'loss': [0.6962836757302284, 0.6937669888138771]}

In [25]:
model.predict(x_train)

array([[[0.5       ],
        [0.5331072 ],
        [0.5       ],
        ...,
        [0.5       ],
        [0.5       ],
        [0.5       ]],

       [[0.5       ],
        [0.5099149 ],
        [0.5       ],
        ...,
        [0.5       ],
        [0.5       ],
        [0.5       ]],

       [[0.5       ],
        [0.53613913],
        [0.5       ],
        ...,
        [0.5       ],
        [0.5       ],
        [0.5       ]],

       ...,

       [[0.5       ],
        [0.5262796 ],
        [0.5       ],
        ...,
        [0.5       ],
        [0.5       ],
        [0.5       ]],

       [[0.5       ],
        [0.51714003],
        [0.5       ],
        ...,
        [0.5       ],
        [0.5       ],
        [0.5       ]],

       [[0.5       ],
        [0.537264  ],
        [0.5       ],
        ...,
        [0.5       ],
        [0.5500683 ],
        [0.5389925 ]]], dtype=float32)

In [ ]:
def init_encoder(input_shape=(10,1), latent_dim=2):
    encoder_input = Input(shape=input_shape)
    n_nodes = np.prod(input_shape)

    # Convolution
    h = encoder_input
    h = Reshape((timesteps, notes))(h)
    h = Conv1D(
        64, kernel_size=2, strides=1, activation='relu', padding='valid')(h)

    h = Bidirectional(LSTM(128))(h)

    # Z Mean, Variance
    z_mean = Dense(latent_dim, name='z_mean')(h)  # no activation='relu'
    z_log_var = Dense(latent_dim, name='z_log_var')(h)  # no activation='relu'

    encoder_output = [z_mean, z_log_var]
    encoder_model = Model(encoder_input, encoder_output, name='encoder_model-')
    return encoder_model, encoder_input, z_mean, z_log_var


def init(input_shape=(36, 1), latent_dim=10, epsilon_std=1.):
    encoder_model, encoder_input, z_mean, z_log_var = encoder(
        input_shape, latent_dim)
    z_input = encoder_model(encoder_input)

    # this function must be defined locally in order for the model to be serializable
    # using the following inline lambda function will result in an error when .to_json() is called
    #   lambda args: sample(args, z_mean, z_log_var, latent_dim, epsilon_std)
    def sample(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(
            shape=(K.shape(z_mean)[0], latent_dim),
            mean=0.,
            stddev=epsilon_std)
        return z_mean + K.exp(z_log_var) * epsilon

    z_output = Lambda(sample)(z_input)
    decoder_model = decoder(z_output, latent_dim, input_shape)

    # VAE
    vae_input = Input(shape=(input_shape, ))
    vae_input = encoder_input
    vae_output = decoder_model(z_output)
    # vae_output = decoder_model(z_mean)
    vae = Model(vae_input, vae_output, name='vae-model-')

    vae_loss_ = vae_loss(
        vae_input,
        vae_output,
        z_mean,
        z_log_var,
        timesteps,
        notes,
        beta=0.75,
        gamma=0.05)
    vae.add_loss(vae_loss_)
    vae.compile(optimizer='adam')

    # Encoder (matrices -> latent vectors)
    encoder_ = Model(encoder_input, z_mean)
    # Generator (latent vectors -> matrices)
    # generator_input = Input((latent_dim, ))
    # generator_layers_ = utils.composition(decoders, generator_input)
    # generator = Model(generator_input, generator_layers_)
    return vae, encoder_, decoder_model


https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
    
<img src="https://cdn-images-1.medium.com/max/1200/1*BrwllztzxZO89qCB9w5d-w.png" style="width:50%;"/>